## Fidelity and First Order Conditions

#### Imports

In [1]:
# using Pkg
# Pkg.activate(".")
# # Pkg.rm("QuantumCollocation")
# Pkg.develop(path="../../QuantumCollocation.jl")
# using Revise
using Piccolo
using LinearAlgebra
using PiccoloQuantumObjects

#### Setup

In [10]:
# set time parameters
T = 50
Δt = 0.2
H_drive = PAULIS.X
U_goal = GATES.X
rob_scale = 1 / 8.0
ϵ = 0
piccolo_opts = PiccoloOptions(verbose=false)

2×2 Matrix{ComplexF64}:
 0.0+0.0im  0.5+0.0im
 0.5+0.0im  0.0+0.0im

## Multiplicative Error System


In [ ]:
H_drive_mult(ϵ) = (1 + ϵ) * H_drive
∂ₑHₘ = H_drive
varsys_mult = VariationalQuantumSystem([H_drive_mult(0)], [∂ₑHₘ])

prob_mult_rob = UnitaryVariationalProblem(
    varsys_mult, U_goal, T, Δt;
    variational_scales=[rob_scale],
    robust_times=[[T]],
    piccolo_options=piccolo_opts
)

solve!(prob_mult_rob, max_iter=50, print_level=0)

    constructing UnitarySmoothPulseProblem...
	using integrator: typeof(UnitaryIntegrator)
	control derivative names: [:da, :dda]
	applying timesteps_all_equal constraint: Δt
    initializing optimizer...
        applying constraint: timesteps all equal constraint
        applying constraint: initial value of Ũ⃗
        applying constraint: initial value of a
        applying constraint: final value of a
        applying constraint: bounds on a
        applying constraint: bounds on da
        applying constraint: bounds on dda
        applying constraint: bounds on Δt
This is Ipopt version 3.14.17, running with linear solver MUMPS 5.8.0.

Number of nonzeros in equality constraint Jacobian...:     5502
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:   156403

Total number of variables............................:      738
                     variables with only lower bounds:        0
                variables with 

## Additive error system

In [5]:
H_drive = PAULIS.X
H_drift(ϵ) = ϵ * PAULIS.X
∂ₑHₐ = PAULIS.X
varsys_add = VariationalQuantumSystem(H_drift(0), [H_drive_add], [∂ₑHₐ])

prob_add_rob = UnitaryVariationalProblem(
    varsys_add, U_goal, T, Δt;
    variational_scales=[rob_scale],
    piccolo_options=piccolo_opts
)

solve!(prob_add_rob, max_iter=50, print_level=0)

Fidelity: 0.9999984399524835


(2, 50)

## Comparison

In [11]:
mult_rob_n = norm(rob_scale * prob_mult_rob.trajectory.Ũ⃗ᵥ1) |> println


println("Multiplcative control: ")
size(prob_mult_rob.trajectory.a) |> println
println("Multiplcative unitary: ")
size(prob_mult_rob.trajectory.Ũ⃗) |> println

println("Additive control: ")
size(prob_add_rob.trajectory.a) |> println
println("Additive unitary: ")
size(prob_add_rob.trajectory.Ũ⃗) |> println


iso_vec_to_operator(prob_mult_rob.trajectory.Ũ⃗[:, end]) |> pretty_print
iso_vec_to_operator(prob_add_rob.trajectory.Ũ⃗[:, end]) |> pretty_print

println("Final fidelity for multiplicative error: ", unitary_rollout_fidelity(prob_mult_rob.trajectory, varsys_mult))
println("Final fidelity additive error: ", unitary_rollout_fidelity(prob_add_rob.trajectory, varsys_add))


    constructing UnitarySmoothPulseProblem...
	using integrator: typeof(UnitaryIntegrator)
	control derivative names: [:da, :dda]
	applying timesteps_all_equal constraint: Δt
    initializing optimizer...
        applying constraint: timesteps all equal constraint
        applying constraint: initial value of Ũ⃗
        applying constraint: initial value of a
        applying constraint: final value of a
        applying constraint: bounds on a
        applying constraint: bounds on da
        applying constraint: bounds on dda
        applying constraint: bounds on Δt
This is Ipopt version 3.14.17, running with linear solver MUMPS 5.8.0.

Number of nonzeros in equality constraint Jacobian...:     5502
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     2739

Total number of variables............................:      738
                     variables with only lower bounds:        0
                variables with 